# Interactive Spectogram Viewer

### Original kernel by [@linainversez](https://www.kaggle.com/linainversez): [spectrogram-viewer](https://www.kaggle.com/linainversez/spectrogram-viewer) 

### Run in interactive mode to view spectrograms

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ipywidgets import interact
from ipywidgets import widgets
from IPython.display import display

In [ ]:
# TRAIN_FILES = glob.glob( '../input/birdclef-2021/train_short_audio/*/*.ogg' )
# df = pd.read_csv('../input/birdclef-2021/train_metadata.csv')
TRAIN_FILES = glob.glob( '../input/birdclef-2022/train_audio/*/*.ogg' )
df = pd.read_csv('../input/birdclef-2022/train_metadata.csv')

# fft params
SR = 32000
N_FFT = 2048 #1024
HOP_LEN = 512
FMIN = 20
FMAX = 16000
# CLIP_LENGTH = 5 # 7 # number of seconds to clip around the sound
# START_TIME = 1  # start 2 seconds before t_min


scored_birds = [
    "akiapo", "aniani", "apapan", "barpet", "crehon", "elepai", "ercfra", 
    "hawama", "hawcre", "hawgoo", "hawhaw", "hawpet1", "houfin", "iiwi", 
    "jabwar", "maupar", "omao", "puaioh", "skylar", "warwhe1", "yefcan"
]

### If you want to see ONLY SCORED birds 
BIRD_CODE = {p:i for i,p in enumerate(sorted(scored_birds))}
INV_BIRD_CODE = {i:p for i,p in enumerate(sorted(scored_birds))}
N_CLASS = len(BIRD_CODE) # 21

### If you want to see ALL birds uncoment below
# BIRD_CODE = {p:i for i,p in enumerate(sorted(df.primary_label.unique()))}
# INV_BIRD_CODE = {i:p for i,p in enumerate(sorted(df.primary_label.unique()))}
# N_CLASS = len(BIRD_CODE) # 152


print('no of audios:', len(TRAIN_FILES) ) # 62874
print('no of classes:', N_CLASS ) # 152

## Helper Functions

In [ ]:
# def get_full_path(bird_class, recording_id):
#     return f'../input/birdclef-2021/train_short_audio/{bird_class}/{recording_id}'

# def pad_truncate_sequence(x, max_len):
#     if len(x) < max_len:
#         return np.concatenate((x, np.zeros(max_len - len(x)))) # pad back
#     else:
#         return x[0 : max_len]

# def cut_audio(audio, t_min, start_time=START_TIME):
#     start = max(0, t_min - start_time)
#     end = start + CLIP_LENGTH
#     snippet = audio[int(start*SR):int(end*SR)]
#     if t_min - start_time < 0:
#         return t_min, pad_truncate_sequence(snippet, CLIP_LENGTH*SR)
#     else:
#         return START_TIME, pad_truncate_sequence(snippet, CLIP_LENGTH*SR)

def get_full_path(bird_class, recording_id):
    return f'../input/birdclef-2022/train_audio/{recording_id}'


def load_audio(bird_class, recording_id):
    audio, _ = librosa.core.load(get_full_path(bird_class, recording_id), sr=SR, mono=True)
    return audio

def create_spectrogram_from_row(row):
    recording_id = row.filename
    bird_class = row.primary_label
    audio = load_audio(bird_class, recording_id)
    #start, audio = cut_audio(audio, t_min)
    stft = librosa.stft(audio, n_fft=N_FFT, hop_length=HOP_LEN, win_length=N_FFT)
    S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    fig, ax = plt.subplots(figsize=(15,6))
    ax.set_title(f"[Bird: {bird_class}] Spectrogram of Audio:{recording_id.split('/')[-1]}", fontsize=15)
    img = librosa.display.specshow(S_db, hop_length=HOP_LEN, sr=SR, fmin=FMIN, fmax=FMAX, x_axis='time', y_axis='hz')  # ['linear', 'log', 'mel']
    # ax.add_patch(patches.Rectangle(xy=(start, f_min), width=t_max-t_min, height=f_max-f_min, ec='#00ff00', fill=False) )
    fig.colorbar(img, ax=ax)

In [ ]:
def update_x_range(*args):
    image_slider.max = df[df.primary_label == INV_BIRD_CODE[int(species_dropdown.value)]].shape[0]-1
    
image_slider = widgets.IntSlider(min=0, max=df[df.primary_label == INV_BIRD_CODE[0]].shape[0]-1, value=0)
species_dropdown = widgets.Dropdown(options=list(range(N_CLASS)), value=0) ## BIRD_CODE    
image_slider.observe(update_x_range, 'value')

def view_image(audio_idx, bird_id):
    sub_df = df[df.primary_label == INV_BIRD_CODE[int(bird_id)]]
    create_spectrogram_from_row(sub_df.iloc[audio_idx])

# Spectogram Viewer

In [ ]:
#### Run in interactive mode to view spectrograms

## audio_idx: no of audio file for selected bird 
## bird_id: bird id class 0-20 or 0-151, see INV_BIRD_CODE

interact(view_image, audio_idx=image_slider, bird_id=species_dropdown);

In [ ]:
INV_BIRD_CODE